In [1]:
import pandas as pd
from constants import pool_addresses, w3, latest_block, WETH
from get_prices import get_eth_price, get_token_to_eth
token_df = pd.read_csv('token_df.csv', index_col=0)
timestamp_df = pd.read_csv('timestamp_df.csv', index_col=0)
eth_price_df = pd.read_csv('eth_price_df.csv', index_col=0)
eth_price_df

,price,timestamp,block_number,token_address
0,1566.867312,1667260800,15871479,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
1,1584.923880,1667174400,15864318,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
2,1611.743814,1667088000,15857155,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
3,1552.394682,1667001600,15850001,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
4,1506.774642,1666915200,15842846,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
...,...,...,...,...
175,2226.751183,1652140800,14745349,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
176,2508.533551,1652054400,14739153,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
177,2629.164695,1651968000,14732824,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
178,2687.325123,1651881600,14726562,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2


In [10]:
from pool import Pool, build_historial_df
from get_prices import build_asset_il_df

historical_dfs = []
il_dfs = []
for p in pool_addresses:
    try:
        df = build_historial_df(Pool(p), timestamp_df, eth_price_df)
        il_df = build_asset_il_df(df, 7)
        il_dfs.append(il_df)
        historical_dfs.append(df)
    except Exception as e:
        print(e, type(e), p)


BadFunctionCallOutput: Could not decode contract function call to getReserves with return data: b'', output_types: ['uint112', 'uint112', 'uint32']

In [18]:
il_df = pd.concat(il_dfs)
# 50th Percentile
def q50(x): # 50th perc
    return x.quantile(0.5)
# 90th Percentile

def q90(x):
    return x.quantile(0.9)

def q10(x):
    return x.quantile(0.1)

descriptive_stats_il_by_pool = il_df.groupby(['pool_address'])['il'].agg(
    ['mean', 'std', 'median', 'min', 'max', q10, q90])
descriptive_stats_il_by_pool.sort_values('min')

,mean,std,median,min,max,q10,q90
pool_address,,,,,,,
0xe55c3e83852429334A986B265d03b879a3d188Ac,-0.954635,2.046706,-0.186143,-17.255608,-1.958914e-05,-2.741641,-0.010989
0xdC08159A6C82611aEB347BA897d82AC1b80D9419,-0.617989,1.449010,-0.028706,-10.916988,0.000000e+00,-2.005017,0.000000
0xecBa967D84fCF0405F6b32Bc45F4d36BfDBB2E81,-0.416449,0.984553,-0.089215,-6.681955,-1.121144e-04,-0.742259,-0.003500
0x61eB53ee427aB4E007d78A9134AaCb3101A2DC23,-0.418503,1.000308,-0.090662,-6.603694,-2.401961e-05,-0.744402,-0.003173
0x43AE24960e5534731Fc831386c07755A2dc33D47,-0.256851,0.655647,-0.053744,-4.384190,-2.240838e-05,-0.493682,-0.003529
0xA1d7b2d891e3A1f9ef4bBC5be20630C2FEB1c470,-0.257764,0.654261,-0.060241,-4.383796,-3.736071e-06,-0.503513,-0.001889
0xAd5B1a6ABc1C9598C044cea295488433a3499eFc,-0.163116,0.456914,-0.040359,-3.195315,-3.006028e-05,-0.260543,-0.000913
0xC3f279090a47e80990Fe3a9c30d24Cb117EF91a8,-0.122128,0.224880,-0.033383,-1.746782,-2.046439e-06,-0.317980,-0.000756
0x53162D78dCa413d9e28cf62799D17a9E278B60E8,-0.177930,0.305964,-0.053146,-1.497316,-9.614350e-07,-0.464861,-0.002204


Everything but 
- APWine: APW Token https://etherscan.io/address/0x4104b135DBC9609Fc1A9490E61369036497660c8 
- Mycelium https://etherscan.io/token/0x4b13006980aCB09645131b91D259eaA111eaF5Ba
fail with `<class 'web3.exceptions.ContractLogicError'> execution reverted`

I'm not sure why it fails but we can infer the price with the ratio of assets in the APW Token, WETH pool. 


In [ ]:
# compute_il(1.160473,2.265778,1948.387988, 2735.270596)
# spot checked with https://dailydefi.org/tools/impermanent-loss-calculator/

# How do I think about the IL risk of a pool

1. For the last 6 months, in week long increments what was the distribution of historical IL?
- How stable is this distribtuion over time
- What is the relationship between mean, std and 90th percentile IL between the weeks.

2. How stable is the month wise corrilation between prices of the tokens.
- What is the corr from prices.


I want a dataframe of

day, asset1 amount, asset 2 amount, asset1 over asset 2



In [ ]:
from pool import Pool
for p in pool_addresses:
    pool = Pool(p)
    if WETH in [pool.token0_address, pool.token1_address]:
        pass
    else:
        print('not', p)
# because all of the pools have